In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# Unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf /content/gdrive/My\ Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/ #gabriela
#!tar -xzvf /content/gdrive/My\ Drive/Bees/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/ #pascal

!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.6.5
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 5
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


In [ ]:
print(cv2.__version__)

4.1.2


# Start validadion

In [3]:
#!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import sqlite3
from shapely.geometry import Polygon
from copy import deepcopy
import pandas as pd
import seaborn as sns
import imutils
import time
from time import gmtime, strftime

import os.path
from numpy import genfromtxt

import pandas as pd

import xml.etree.ElementTree as ET


## Functions

In [4]:
import xml.etree.ElementTree as xmlbuilder
import xml.dom.minidom


def create_xml_voc_files(bboxes, gt_bboxes, img_sizes, xml_folder_path):
  root = xmlbuilder.Element("annotation")
  fldr = xmlbuilder.Element("folder")
  fldr.text = "validate"
  root.append(fldr)

  for img_file in bboxes.keys():
    xml_filename = xml_folder_path + img_file.split('.')[0] + '.xml'

    #exit if file exists
    if os.path.isfile(xml_filename):
      return

    flnm = xmlbuilder.Element("filename")
    flnm.text = img_file
    root.append(flnm)
    pth = xmlbuilder.Element("path")
    pth.text = img_file
    root.append(pth)

    src = xmlbuilder.Element("source")
    db = xmlbuilder.SubElement(src, "database")
    db.text = "Unknown"
    root.append(src)

    sz = xmlbuilder.Element("size")
    wd = xmlbuilder.SubElement(sz, "width")
    wd.text = str(img_sizes[0][0])
    hg = xmlbuilder.SubElement(sz, "height")
    hg.text = str(img_sizes[0][1])
    dp = xmlbuilder.SubElement(sz, "depth")
    dp.text = str(3)
    root.append(sz)

    for box in bboxes[img_file]['boxes']:
      detected_bee = xmlbuilder.Element("object")
      db_name = xmlbuilder.SubElement(detected_bee, "name")
      db_name.text = "detected_bee"
      db_pose = xmlbuilder.SubElement(detected_bee, "pose")
      db_pose.text = "Unspecified"
      db_truncated = xmlbuilder.SubElement(detected_bee, "truncated")
      db_truncated.text = str(0)
      db_difficult = xmlbuilder.SubElement(detected_bee, "difficult")
      db_difficult.text = str(0)
      db_bndbox = xmlbuilder.SubElement(detected_bee, "bndbox")
      db_bndbox_xmin = xmlbuilder.SubElement(db_bndbox, "xmin")
      db_bndbox_xmin.text = str(box[0])
      db_bndbox_ymin = xmlbuilder.SubElement(db_bndbox, "ymin")
      db_bndbox_ymin.text = str(box[1])
      db_bndbox_xmax = xmlbuilder.SubElement(db_bndbox, "xmax")
      db_bndbox_xmax.text = str(box[2])
      db_bndbox_ymax = xmlbuilder.SubElement(db_bndbox, "ymax")
      db_bndbox_ymax.text = str(box[3])
      root.append(detected_bee)

    for box in gt_bboxes[img_file]:
      gt_bee = xmlbuilder.Element("object")
      db_name = xmlbuilder.SubElement(gt_bee, "name")
      db_name.text = "gt_bee"
      db_pose = xmlbuilder.SubElement(gt_bee, "pose")
      db_pose.text = "Unspecified"
      db_truncated = xmlbuilder.SubElement(gt_bee, "truncated")
      db_truncated.text = str(0)
      db_difficult = xmlbuilder.SubElement(gt_bee, "difficult")
      db_difficult.text = str(0)
      db_bndbox = xmlbuilder.SubElement(gt_bee, "bndbox")
      db_bndbox_xmin = xmlbuilder.SubElement(db_bndbox, "xmin")
      db_bndbox_xmin.text = str(box[0])
      db_bndbox_ymin = xmlbuilder.SubElement(db_bndbox, "ymin")
      db_bndbox_ymin.text = str(box[1])
      db_bndbox_xmax = xmlbuilder.SubElement(db_bndbox, "xmax")
      db_bndbox_xmax.text = str(box[2])
      db_bndbox_ymax = xmlbuilder.SubElement(db_bndbox, "ymax")
      db_bndbox_ymax.text = str(box[3])
      root.append(gt_bee)

    tree = xmlbuilder.ElementTree(root) 
    
    xmlstr = xml.etree.ElementTree.tostring(root, encoding='utf8', method='xml')
    dom = xml.dom.minidom.parseString(xmlstr) #xml_string
    pretty_xml_as_string = dom.toprettyxml()

    with open (xml_filename, "w") as files : 
        files.write("%s\n" % pretty_xml_as_string)


In [5]:
def CNN_get_boxes_for_frame(file,image):

  predicted = {}
  predicted[file] = {}
  predicted[file]['boxes'] = []
  predicted[file]['scores'] = []
  bboxes = []
  (W, H) = (None, None)
  no_predictions = False

  # image = frame

  # construct a blob from the input frame and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes
  # and associated probabilities
  blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (608, 608), swapRB=True, crop=False)
  net.setInput(blob)
  start = time.time()
  layerOutputs = net.forward(ln)

  if W is None or H is None:
    (H, W) = image.shape[:2]

  # loop over each of the layer outputs
  for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
      # extract the class ID and confidence (i.e., probability)
      # of the current object detection
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]
      # filter out weak predictions by ensuring the detected
      # probability is greater than the minimum probability
      if confidence > 0.5:
        # scale the bounding box coordinates back relative to
        # the size of the image, keeping in mind that YOLO
        # actually returns the center (x, y)-coordinates of
        # the bounding box followed by the boxes' width and
        # height
        box = detection[0:4] * np.array([W, H, W, H])
        (centerX, centerY, width, height) = box.astype("int")
        # use the center (x, y)-coordinates to derive the top
        # and and left corner of the bounding box
        x_min = int(centerX - (width / 2))
        x_max = x_min+width
        y_min = int(centerY - (height / 2))
        y_max = y_min+height
        # update our list of bounding box coordinates,
        # confidences, and class IDs
        #boxes.append([x, y, int(width), int(height)])
        #confidences.append(float(confidence))
        #classIDs.append(classID)
        #boxes_to_frames.append([framesCaptured,x, y, int(width), int(height),confidence])
        predicted[file]['boxes'].append([x_min, y_min, x_max, y_max])
        predicted[file]['scores'].append(float(confidence))
  if len(predicted[file]['boxes']) == 0:
    no_predictions = True
  return predicted, no_predictions

In [6]:
def plot_image_and_boxes_on_axis(image, boxes, ax, centers=True, trace=True, title="",color='blue'):
    ax.imshow(image[:, :, ::-1])
    for i, box_dict in enumerate(boxes):
        # box = box_dict['location']
        box = box_dict
        rect = patches.Rectangle((box[0], box[1]), box[2], box[3], fill=False, lw=2, color=color)

        ax.add_patch(rect)
        ax.set_axis_off()
    ax.title.set_text(title)
    return ax

In [7]:
def calc_iou(gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and
    return the IoU ratio
    '''
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt = gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p = pred_bbox

    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt > y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p > y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct", x_topleft_p, x_bottomright_p, y_topleft_p,
                             y_bottomright_p)

    # if the GT bbox and predcited BBox do not overlap then iou=0
    if (x_bottomright_gt < x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox

        return 0.0
    if (
            y_bottomright_gt < y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox

        return 0.0
    if (
            x_topleft_gt > x_bottomright_p):  # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox

        return 0.0
    if (
            y_topleft_gt > y_bottomright_p):  # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox

        return 0.0

    GT_bbox_area = (x_bottomright_gt - x_topleft_gt + 1) * (y_bottomright_gt - y_topleft_gt + 1)
    Pred_bbox_area = (x_bottomright_p - x_topleft_p + 1) * (y_bottomright_p - y_topleft_p + 1)

    x_top_left = np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])

    intersection_area = (x_bottom_right - x_top_left + 1) * (y_bottom_right - y_top_left + 1)

    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)

    return intersection_area / union_area


In [8]:
def get_model_scores(pred_boxes):
    """Creates a dictionary of from model_scores to image ids.
    Args:
        pred_boxes (dict): dict of dicts of 'boxes' and 'scores'
    Returns:
        dict: keys are model_scores and values are image ids (usually filenames)
    """
    model_score={}
    for img_id, val in pred_boxes.items():
        for score in val['scores']:
            if score not in model_score.keys():
                model_score[score]=[img_id]
            else:
                model_score[score].append(img_id)
    return model_score

In [9]:
def calc_precision_recall(image_results):
    """Calculates precision and recall from the set of images
    Args:
        img_results (dict): dictionary formatted like:
            {
                'img_id1': {'true_pos': int, 'false_pos': int, 'false_neg': int},
                'img_id2': ...
                ...
            }
    Returns:
        tuple: of floats of (precision, recall)
    """
    true_positive=0
    false_positive=0
    false_negative=0
    precision = 0
    recall = 0
    for img_id, res in image_results.items():
        true_positive +=res['true_positive']
        false_positive += res['false_positive']
        false_negative += res['false_negative']
        try:
            precision = true_positive/(true_positive+ false_positive)
        except ZeroDivisionError:
            precision=0.0
        try:
            recall = true_positive/(true_positive + false_negative)
        except ZeroDivisionError:
            recall=0.0
    return (precision, recall)


In [10]:
def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.
    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.
    Returns:
        dict: qty_ground_truth (int), qty_predicted (int), true positives (int), false positives (int), false negatives (int)
    """
    all_pred_indices = range(len(pred_boxes))
    all_gt_indices = range(len(gt_boxes))
    qty_predicted = len(pred_boxes)
    qty_ground_truth = len(gt_boxes)
    tp = 0
    fp = 0
    fn = 0
    output = "output"

    if len(all_pred_indices) == 0:
        return {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': fp, 'false_negative': qty_ground_truth}
    if len(all_gt_indices) == 0:
        return {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': qty_predicted, 'false_negative': fn}

    gt_idx_thr = []
    pred_idx_thr = []
    ious = []

    for ipb, pred_box in enumerate(pred_boxes):
        # print(ipb, pred_box)

        for igb, gt_box in enumerate(gt_boxes):
            # print(igb, gt_box)
            iou = calc_iou(gt_box, pred_box)

            if iou > iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)
    iou_sort = np.argsort(ious)[::1]
    if len(iou_sort) == 0:
        return {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': len(pred_boxes), 'false_negative': fn}
    else:
        gt_match_idx = []
        pred_match_idx = []
        for idx in iou_sort:
            gt_idx = gt_idx_thr[idx]
            pr_idx = pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if (gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp = len(gt_match_idx)
        fp = len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)
        output = {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': fp, 'false_negative': fn}
    return output

In [11]:
def get_avg_precision_at_iou(image,gt_boxes, pred_bb, no_predictions = False, iou_thr=0.5):
  if (no_predictions):
    return {image:{'qty_ground_truth' : len(gt_boxes[image]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[image])}}

  model_scores = get_model_scores(pred_bb)
  # TODO: does not work with scores that are all equal
  #print("model_scores: ", model_scores)
  sorted_model_scores = sorted(model_scores.keys())
  # print("ggg sorted model scores: ", sorted_model_scores)
  # Sort the predicted boxes in descending order (lowest scoring boxes first):
  for img_id in pred_bb.keys():
        #print('img_id: ',img_id)
        arg_sort = np.argsort(pred_bb[img_id]['scores'])
        pred_bb[img_id]['scores'] = np.array(pred_bb[img_id]['scores'])[arg_sort].tolist()
        pred_bb[img_id]['boxes'] = np.array(pred_bb[img_id]['boxes'])[arg_sort].tolist()

  pred_boxes_pruned = deepcopy(pred_bb)

  precisions = []
  recalls = []
  model_thrs = []
  img_results = {}
  img_ids = []
  model_score_thr = 0
  # Loop over model score thresholds and calculate precision, recall
  for ithr, model_score_thr in enumerate(sorted_model_scores[:-1]):
    # On first iteration, define img_results for the first time:
    # print("Model score thr : ", model_score_thr)
    img_ids = gt_boxes.keys() if ithr == 0 else model_scores[model_score_thr]

  if len(img_ids) == 0:
    print("pruned img_ids from after score", image)
    return {image:{'qty_ground_truth' : len(gt_boxes[image]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[image])}}

  for img_id in img_ids:
    # return if the size of prediction is 0
    if len(pred_bb[img_id]['boxes']) == 0:
      return {img_id:{'qty_ground_truth' : len(gt_boxes[img_id]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[img_id])}}

    gt_boxes_img = gt_boxes[img_id]
    box_scores = pred_boxes_pruned[img_id]['scores']
    start_idx = 0

    for score in box_scores:
      if score <= 0:
        pred_boxes_pruned[img_id]
        start_idx += 1
      else:
        break
        # Remove boxes, scores of lower than threshold scores:
    pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
    pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]
    # Recalculate image results for this image
    img_results[img_id] = get_single_image_results(gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], iou_thr=0.25)
    # calculate precision and recall
    
    #if len(img_results) == 0 and len(pred_bb[img_id]['boxes']) == 0 and len(gt_boxes[img_id]) != 0:
    if len(img_results[img_id]) == 0 and len(gt_boxes[img_id]) != 0:
      print("get_avg_precision_at_iou all predictions were pruned")
      return {img_id:{'qty_ground_truth' : len(gt_boxes[img_id]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[img_id])}}

  prec, rec = calc_precision_recall(img_results)
  precisions.append(prec)
  recalls.append(rec)
  model_thrs.append(model_score_thr)
  precisions = np.array(precisions)
  recalls = np.array(recalls)
  prec_at_rec = []
  for recall_level in np.linspace(0.0, 1.0, 11):
    try:
      args = np.argwhere(recalls > recall_level).flatten()
      prec = max(precisions[args])
      #print(recalls, "Recall")
      #print(recall_level, "Recall Level")
      #print(args, "Args")
      #print(prec, "precision")
    except ValueError:
      prec = 0.0
    prec_at_rec.append(prec)
  avg_prec = np.mean(prec_at_rec)
  return img_results

In [12]:
#  Felzenszwalb et al.
def non_max_suppression_slow(boxes, overlapThresh=0.3):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# initialize the list of picked indexes
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
 	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:

		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]
    # loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]
			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])
			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)
			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]
			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)
		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)
	# return only the bounding boxes that were picked
	return pick

In [13]:
# load the predictions and ground_truth from already saved images, respectivelly from their xml files
def load_boxes_from_xmls(validated_xml_directory_path, display_results=False):
    results = []
    for hive_model,xml_files_path in validated_xml_directory_path.items():
        print('hive_model:',hive_model)
        print('xml_files_path:',xml_files_path)

        xml_lst = []
        for filename in os.listdir(xml_files_path):
            if not filename.endswith('.xml'): continue
            fullname = os.path.join(xml_files_path, filename)
            xml_lst.append(fullname)

        # work on single image/xml
        for xml in xml_lst:
            etree = ET.parse(xml)
            root = etree.getroot()

            img_name = root.find('filename').text

            predicted = {}
            predicted[img_name] = {}
            predicted[img_name]['boxes'] = []
            predicted[img_name]['scores'] = []
            gt_bboxes = {}
            no_predictions = False

            p_bbxcoords = []
            gt_pbbxcoords = []
            scores = []

            objects = root.findall('object')
            score = 0.70

            for obj in objects:
                box = obj.find('bndbox')
                xmin = int(box.find('xmin').text)
                ymin = int(box.find('ymin').text)
                xmax = int(box.find('xmax').text)
                ymax = int(box.find('ymax').text)

                bbtype = obj.find('name').text

                if bbtype == 'detected_bee':
                    p_bbxcoords.append([xmin, ymin, xmax, ymax])
                    scores.append(score)
                    score += 0.0005
                elif bbtype == 'gt_bee':
                    gt_pbbxcoords.append([xmin, ymin, xmax, ymax])
                else:
                    print('Error object type at ', img_name)

            predicted[img_name]['boxes'] = np.array(p_bbxcoords)
            predicted[img_name]['scores'] = np.array(scores)
            gt_bboxes[img_name] = np.array(gt_pbbxcoords)

            if len(predicted[img_name]['boxes']) == 0:
                no_predictions = True

            #print('img_name: ',img_name, ', nr predicted: ', len(p_bbxcoords), 'nr gt: ', len(gt_pbbxcoords))

            res = get_avg_precision_at_iou(img_name, gt_bboxes, predicted, no_predictions)
            if display_results:
                print(res)
            results.append([hive_model,res])
    return results

In [14]:
# calculate validation results per hive and save to file 
def calc_validation_results(output, trained_model, display_results=True):
  results_per_hive_file_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/validation_results_per_hive.csv'
  print(results_per_hive_file_path)
  results_for_model_file_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/validation_results_for_model.csv'
  print(results_for_model_file_path)

  if os.path.exists(results_per_hive_file_path):
    os.remove(results_per_hive_file_path)
  if os.path.exists(results_for_model_file_path):
    os.remove(results_for_model_file_path)

  total_qty_ground_truth = output['qty_ground_truth'].sum()
  total_qty_predicted = output['qty_predicted'].sum()
  total_true_positive = output['true_positive'].sum()
  total_false_negative = output['false_negative'].sum()
  total_false_positive = output['false_positive'].sum()

  all_names = []
  #for name in output.hive:
  for name in output.hive_name:
    all_names.append(name)

  # Results per hive
  results_per_hive_file = open(results_per_hive_file_path, 'w')
  results_per_hive_file.write("trained_model,hive_name,qty_ground_truth,qty_predicted,true_positive,false_positive,false_negative,nr_imgs,precision,recall\n")

  for hive_name in list(set(all_names)):
    filtered_df = output.loc[output['hive_name'] == hive_name]
    true_positive = filtered_df['true_positive'].sum()
    false_positive = filtered_df['false_positive'].sum()
    false_negative = filtered_df['false_negative'].sum()
    nr_imgs = len(filtered_df)
    precision = true_positive/(true_positive+false_positive)
    recall = true_positive / (true_positive + false_negative)

    if display_results:
      print(hive_name)
      print("TP: {}".format(true_positive))
      print("FP: {}".format(false_positive))
      print("FN: {}".format(false_negative))
      print("{} images".format(nr_imgs))
      print("Precision: {}".format(precision))
      print("Recall: {}".format(recall))
      print("_____________________________________________________________")

    qty_ground_truth = filtered_df['qty_ground_truth'].sum()
    qty_predicted = filtered_df['qty_predicted'].sum()

    line_str = trained_model + ','
    line_str += hive_name + ','
    line_str += str(qty_ground_truth) + ','
    line_str += str(qty_predicted) + ','
    line_str += str(true_positive) + ','
    line_str += str(false_positive) + ','
    line_str += str(false_negative) + ','
    line_str += str(nr_imgs) + ','
    line_str += str(precision) + ','
    line_str += str(recall)
    results_per_hive_file.write("%s\n" % line_str)
  results_per_hive_file.close()

  # Aggregated resulsts for model
  total_precision = total_true_positive/(total_true_positive+total_false_positive)
  total_recall = total_true_positive / (total_true_positive + total_false_negative)

  if display_results:
    #print("Model name: {}".format(WEIGHTS_PATH.split('/')[-1]))
    print("Model name: {}".format(trained_model))
    print("Overall TP: {}".format(total_true_positive))
    print("Overall FN: {}".format(total_false_negative))
    print("Overall FP: {}".format(total_false_positive))
    print("{} images".format(len(output)))
    print("Overall Precision: {}".format(total_precision))
    print("Overall Recall: {}".format(total_recall))
    print("_____________________________________________________________")
    print("_____________________________________________________________")

  results_for_model_file = open(results_for_model_file_path, 'w')
  results_for_model_file.write("trained_model,qty_ground_truth,qty_predicted,true_positive,false_positive,false_negative,nr_imgs,precision,recall\n")
  line_str = trained_model + ','
  line_str += str(total_qty_ground_truth) + ','
  line_str += str(total_qty_predicted) + ','
  line_str += str(total_true_positive) + ','
  line_str += str(total_false_positive) + ','
  line_str += str(total_false_negative) + ','
  line_str += str(len(output)) + ','
  line_str += str(total_precision) + ','
  line_str += str(total_recall)
  results_for_model_file.write("%s\n" % line_str)
  results_for_model_file.close()


## Runtime

In [ ]:
print(cv2.__version__)

4.1.2


In [16]:
hives = ['Chueried_Hempbox', 
        'Chueried_Hive01', 'Chueried_Hive01_red70', 'Chueried_Hive01_green80', 
        'ClemensRed', 'ClemensYellow',
        'Doettingen_Hive1', 
        'Echolinde', 'Echolinde_Night', 
        'Erlen_Hive04_diagonalview', 'Erlen_Hive04_frontview', 'Erlen_Hive04_smartphone', 
        'Erlen_Hive11', 'Erlen_Hive11_Night', 'Erlen_Hive11_grayscale', 'Erlen_Hive11_red70',
        'Froh14', 'Froh23_TreeCavity', 'UnitedQueens']

In [17]:
hive_model = 'single_hive/all_hives_trainP'
trained_model = 'single_hive/all_hives_trainP' #'staged_2/Erlen_Hive11_UnitedQueens' #'individual_training/Chueried_Hive01' #'single_hive/ClemensYellow'
WEIGHTS_PATH = '/content/gdrive/My Drive/yolo_models/backup_yolo4/' + trained_model  + '/yolov4_bees_best.weights'
CONFIG_PATH = '/content/gdrive/My Drive/darknet/yolov4_bees_leaky.cfg'
labelsPath = '/content/gdrive/My Drive/darknet/bee.names'  
DB_PATH = '/content/gdrive/My Drive/yolo4_validation/bees.db'
PLOT = True #for local testing to inspect images
VALIDATION_XML_DIRECTORY_PATH_PREFIX = '/content/gdrive/My Drive/yolo4_validation/'


results_file_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/validation_results.csv'
print(results_file_path)

overwrite_xmls = False

/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/validation_results.csv


In [18]:
# if not-overwrite only load images from hives that have not already been validated, meaning, the xml files don't exist, the exisitng xmls/images are saved in a separated structure
# if overwrite, all files in the same structure
image_list = []
validation_xml_directory_path = {}
validated_xml_directory_path = {}

for hive in hives:
    write_folder = True
    dp = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/' + hive
    if not os.path.exists(dp):
        validation_xml_directory_path[hive] = dp
        print("created validation folder: ", validation_xml_directory_path[hive])
        os.makedirs(validation_xml_directory_path[hive])
    else:
        print("folder already exists: ", dp)
        write_folder = overwrite_xmls
        if write_folder:
            validation_xml_directory_path[hive] = dp
        else:
            validated_xml_directory_path[hive] = dp
  
    img_path = '/content/gdrive/My Drive/beeTracking_img_labeling/' + hive + '/validate'
    for filename in os.listdir(img_path):
        if not filename.endswith('.jpg'): continue
        fullname = os.path.join(img_path, filename)
        if write_folder:
            image_list.append(fullname)


created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hempbox
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hive01
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hive01_red70
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hive01_green80
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/ClemensRed
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/ClemensYellow
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Doettingen_Hive1
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Echolinde
created validation folder:  /content/gdrive/My Drive/yolo4_validation/single_

In [19]:
len(image_list)

480

In [20]:
print(validation_xml_directory_path)

{'Chueried_Hempbox': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hempbox', 'Chueried_Hive01': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hive01', 'Chueried_Hive01_red70': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hive01_red70', 'Chueried_Hive01_green80': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Chueried_Hive01_green80', 'ClemensRed': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/ClemensRed', 'ClemensYellow': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/ClemensYellow', 'Doettingen_Hive1': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Doettingen_Hive1', 'Echolinde': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Echolinde', 'Echolinde_Night': '/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/Echolinde_Night', 'Erlen_Hive0

In [ ]:
print(validated_xml_directory_path)

{}


In [21]:
# load the class labels our YOLO model was trained on
LABELS = open(labelsPath).read().strip().split("\n")
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
# derive the paths to the YOLO weights and model configuration

In [22]:
# load our YOLO object detector trained on your dataset 
# and determine only the *output* layer names that we need from YOLO
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(CONFIG_PATH, WEIGHTS_PATH)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

[INFO] loading YOLO from disk...


In [23]:
# arguments
args_confidence = 0.5
args_threshold = 0.3

# load images to validate
image_arrays = []

for i in tqdm(range(len(image_list))):
    img = cv2.imread(image_list[i])
    image_arrays.append(img)

100%|██████████| 480/480 [05:52<00:00,  1.36it/s]


In [24]:
print(image_arrays[0].shape)

(1080, 1920, 3)


In [25]:
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

In [ ]:
#conn.close()

In [26]:
query_str = "select hive_name, count(distinct(filename)), count(*) from validation_ground_truth where hive_name in ("
for hive in hives:
  query_str += "\'" + hive + "\',"
query_str = query_str[0:len(query_str)-1]
query_str += ") group by hive_name"

print(query_str)

c.execute(str(query_str))
#print(c.fetchone())

test = c.fetchall()

for row in test:
  print(row)

select hive_name, count(distinct(filename)), count(*) from validation_ground_truth where hive_name in ('Chueried_Hempbox','Chueried_Hive01','Chueried_Hive01_red70','Chueried_Hive01_green80','ClemensRed','ClemensYellow','Doettingen_Hive1','Echolinde','Echolinde_Night','Erlen_Hive04_diagonalview','Erlen_Hive04_frontview','Erlen_Hive04_smartphone','Erlen_Hive11','Erlen_Hive11_Night','Erlen_Hive11_grayscale','Erlen_Hive11_red70','Froh14','Froh23_TreeCavity','UnitedQueens') group by hive_name
('Chueried_Hempbox', 40, 774)
('Chueried_Hive01', 20, 132)
('Chueried_Hive01_green80', 20, 132)
('Chueried_Hive01_red70', 20, 132)
('ClemensRed', 20, 336)
('ClemensYellow', 20, 486)
('Doettingen_Hive1', 40, 1301)
('Echolinde', 20, 921)
('Echolinde_Night', 20, 432)
('Erlen_Hive04_diagonalview', 40, 607)
('Erlen_Hive04_frontview', 40, 570)
('Erlen_Hive04_smartphone', 20, 302)
('Erlen_Hive11', 20, 1632)
('Erlen_Hive11_Night', 20, 788)
('Erlen_Hive11_grayscale', 20, 1632)
('Erlen_Hive11_red70', 20, 1632)
(

In [27]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2020-10-30 14:47:35


In [28]:
# split in two functions:
# 1. make the predictions of the current images
# 2. load the predictions and ground_truth from already saved images, respectivelly from their xml files
# gt_bboxes = dict { truncated_filename : list[[xmin, ymin, xmax, ymax]]}


results = []
PLOT = False

# 1. make the predictions of the current images
for i in tqdm(range(len(image_list)),position=0,leave=True):
    hive_name = image_list[i].split('/')[5]
    filename = image_list[i].split('/')[-1]
    print("Img File: ", filename)
    
    #c.execute("select hive_name from validation_ground_truth where filename = '{fn}' and hive_name = '{hm}'".format(fn=filename, hm=hive_model))
    #hive_name = c.fetchall()[0][0]

    print("MODEL "  , trained_model , '/' , hive_name )

    bboxes, no_predictions = CNN_get_boxes_for_frame(filename, image_arrays[i])
    indices = non_max_suppression_slow(np.array(bboxes[filename]['boxes']))
    bboxes[filename]['boxes'] = np.array(bboxes[filename]['boxes'])[indices].tolist()
    bboxes[filename]['scores'] = np.array(bboxes[filename]['scores'])[indices].tolist()

    print("predicted: ",len(bboxes[filename]['boxes']))

    c.execute("select xmin,ymin,xmax,ymax from validation_ground_truth where filename = '{fn}' and hive_name = '{hn}'".format(fn=filename, hn=hive_name))
    ground_truth = c.fetchall()
    print("actual ground truth: ",len(ground_truth))
    gt_bboxes = {}
    gt_bboxes[filename] = [list(elem) for elem in ground_truth]

    c.execute("select width, height from validation_ground_truth where filename = '{fn}' and hive_name = '{hn}'".format(fn=filename, hn=hive_name))
    img_sizes = c.fetchall()

    xml_folder_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/' + hive_name + '/'
    #print('xml_folder_path', xml_folder_path)
    create_xml_voc_files(bboxes, gt_bboxes, img_sizes, xml_folder_path)

    res = get_avg_precision_at_iou(filename, gt_bboxes, bboxes, no_predictions)
    print(res)
    if PLOT:
        fig, ax = plt.subplots(1, figsize=(20,10))
        plot_bboxes = []
        for box in bboxes[filename]['boxes']:
            plot_bboxes.append([box[0], box[1], box[2]-box[0], box[3]-box[1]])
        plot_gt_bboxes = []
        for box in gt_bboxes[filename]:
            plot_gt_bboxes.append([box[0], box[1], box[2]-box[0], box[3]-box[1]])
        ax = plot_image_and_boxes_on_axis(image_arrays[i], plot_bboxes, ax,color='blue')
        ax = plot_image_and_boxes_on_axis(image_arrays[i], plot_gt_bboxes, ax,color='green')
        plt.show()
    results.append([hive_name,res])



  0%|          | 0/480 [00:00<?, ?it/s]

Img File:  Chueried_Hempbox_frame_1800.jpg
MODEL  single_hive/all_hives_trainP / Chueried_Hempbox


  0%|          | 1/480 [00:04<32:32,  4.08s/it]

predicted:  1
actual ground truth:  20
pruned img_ids from after score Chueried_Hempbox_frame_1800.jpg
{'Chueried_Hempbox_frame_1800.jpg': {'qty_ground_truth': 20, 'qty_predicted': 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': 20}}
Img File:  Chueried_Hempbox_frame_1100.jpg
MODEL  single_hive/all_hives_trainP / Chueried_Hempbox


  0%|          | 2/480 [00:07<30:56,  3.88s/it]

...

predicted:  1
actual ground truth:  2
pruned img_ids from after score Froh_23_20191013_075304_540_M_frame_690.jpg
{'Froh_23_20191013_075304_540_M_frame_690.jpg': {'qty_ground_truth': 2, 'qty_predicted': 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': 2}}
Img File:  Froh_23_20191013_100059_540_M_frame_230.jpg
MODEL  single_hive/all_hives_trainP / Froh23_TreeCavity


100%|█████████▉| 479/480 [26:59<00:03,  3.33s/it]

predicted:  3
actual ground truth:  3
{'Froh_23_20191013_100059_540_M_frame_230.jpg': {'qty_ground_truth': 3, 'qty_predicted': 3, 'true_positive': 3, 'false_positive': 0, 'false_negative': 0}}
Img File:  Froh_23_20191013_150036_540_M_frame_501.jpg
MODEL  single_hive/all_hives_trainP / Froh23_TreeCavity


100%|██████████| 480/480 [27:02<00:00,  3.38s/it]

predicted:  2
actual ground truth:  7
{'Froh_23_20191013_150036_540_M_frame_501.jpg': {'qty_ground_truth': 7, 'qty_predicted': 2, 'true_positive': 2, 'false_positive': 0, 'false_negative': 5}}


In [29]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2020-10-30 15:14:39


In [30]:
old_results = deepcopy(results)

In [31]:
len(results)

480

In [32]:
for k,v in validated_xml_directory_path.items():
    print(k,v)

In [33]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2020-10-30 15:14:39


In [ ]:
# 2. load the predictions and ground_truth from already saved images, respectivelly from their xml files
existing_results = load_boxes_from_xmls(validated_xml_directory_path)
#results.append(existing_results)

In [ ]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

In [ ]:
len(existing_results)

In [ ]:
results.extend(existing_results)

In [ ]:
len(results)

520

In [35]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2020-10-30 15:14:39


In [34]:
# save validation results to file 
if os.path.exists(results_file_path):
  os.remove(results_file_path)

print(results_file_path)
results_file = open(results_file_path, 'w')
results_file.write("trained_model,hive_name,img_file,qty_ground_truth,qty_predicted,true_positive,false_positive,false_negative\n")
for r in results:
  line_str = trained_model + ','
  line_str += r[0] + ','
  for k,v in r[1].items():
    line_str += k + ','
    line_str += str(v['qty_ground_truth']) + ','
    line_str += str(v['qty_predicted']) + ','
    line_str += str(v['true_positive']) + ','
    line_str += str(v['false_positive']) + ','
    line_str += str(v['false_negative'])
  results_file.write("%s\n" % line_str)
results_file.close()

/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/validation_results.csv


# Results

## load results from image-model-hive validation file

In [36]:
#trained_model	hive_name	img_file	qty_ground_truth	qty_predicted	true_positive	false_positive	false_negative
results_from_file = genfromtxt(results_file_path, delimiter=',',dtype="U255,U50,U255,<i8,<i8,<i8,<i8,<i8",encoding="utf8", names = True)

In [37]:
output = pd.DataFrame()
pd.set_option('display.expand_frame_repr', False)

output = pd.DataFrame(results_from_file)

## show model results per hive and save to file

In [38]:
calc_validation_results(output=output, trained_model=trained_model, display_results=False)

/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/validation_results_per_hive.csv
/content/gdrive/My Drive/yolo4_validation/single_hive/all_hives_trainP/validation_results_for_model.csv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in long_scalars


# Recalculate results files for all hives
for results per model and results per hive


In [ ]:
training_types = ['single_hive', 'individual_training']
#'on_qty_of_train_images',
#,'staged_2'

for training_type in training_types:
  for hive in hives:
    trained_model = training_type +  '/' + hive
    validation_xml_directory_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model
    results_file_path = validation_xml_directory_path + '/validation_results.csv'
    if not os.path.exists(validation_xml_directory_path):
      #print("no validation folder for model : ", trained_model)
      continue
    else:
      results_from_file = []
      results_from_file = genfromtxt(results_file_path, delimiter=',',dtype="U255,U50,U255,<i8,<i8,<i8,<i8,<i8",encoding="utf8", names = True)
      output = pd.DataFrame()
      output = pd.DataFrame(results_from_file)
      calc_validation_results(output=output, trained_model=trained_model, display_results=False)


In [ ]:
from os import listdir
from os.path import isfile, isdir, join
training_types = ['on_qty_of_train_images','staged_2']

for training_type in training_types:
  val_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + training_type
  onlydirectories = [f for f in listdir(val_path) if isdir(join(val_path, f))]
  for f in onlydirectories:
    trained_model = training_type +  '/' + f
    validation_xml_directory_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model
    results_file_path = validation_xml_directory_path + '/validation_results.csv'
    results_from_file = genfromtxt(results_file_path, delimiter=',',dtype="U255,U50,U255,<i8,<i8,<i8,<i8,<i8",encoding="utf8", names = True)
    output = pd.DataFrame()
    output = pd.DataFrame(results_from_file)
    calc_validation_results(output=output, trained_model=trained_model, display_results=False)
